In [1]:
import numpy as np
import nibabel as nib
import glob
import os
import sys
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from tifffile import imsave
import tensorflow as tf
from tensorflow.keras.losses import Loss
import segmentation_models_3D as sm
# segmentation_models = '/exp1/experiment1Softmax/segmentation_models/'

2024-03-10 14:29:06.848601: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Segmentation Models: using `tf.keras` framework.


In [2]:
# sys.path.append(segmentation_models)
import tensorflow.keras.backend as K

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

from tensorflow.keras.backend import clear_session
clear_session()

In [3]:
TRAIN_DATASET_PATH = '/data'
#VALIDATION_DATASET_PATH = 'BraTS2020_ValidationData/MICCAI_BraTS2020_ValidationData'

sample_filename = '/data/UCSF-PDGM-0014_nifti/UCSF-PDGM-0014_FLAIR.nii'
sample_filename2 = '/data/UCSF-PDGM-0014_nifti/UCSF-PDGM-0014_T1.nii'
sample_filename3 = '/data/UCSF-PDGM-0014_nifti/UCSF-PDGM-0014_T2.nii'
sample_filename4 = '/data/UCSF-PDGM-0014_nifti/UCSF-PDGM-0014_T1c.nii'
sample_filename_mask = '/data/UCSF-PDGM-0014_nifti/UCSF-PDGM-0014_tumor_segmentation.nii'

test_image_flair=nib.load(sample_filename).get_fdata()
print(test_image_flair.max())
#Scalers are applied to 1D so let us reshape and then reshape back to original shape. 
# test_image_flair=scaler.fit_transform(test_image_flair.reshape(-1, test_image_flair.shape[-1])).reshape(test_image_flair.shape)


test_image_t1=nib.load('/data/UCSF-PDGM-0014_nifti/UCSF-PDGM-0014_T1.nii').get_fdata()
# test_image_t1=scaler.fit_transform(test_image_t1.reshape(-1, test_image_t1.shape[-1])).reshape(test_image_t1.shape)

test_image_t1ce=nib.load('/data/UCSF-PDGM-0014_nifti/UCSF-PDGM-0014_T1c.nii').get_fdata()
# test_image_t1ce=scaler.fit_transform(test_image_t1ce.reshape(-1, test_image_t1ce.shape[-1])).reshape(test_image_t1ce.shape)

test_image_t2=nib.load('/data/UCSF-PDGM-0014_nifti/UCSF-PDGM-0014_T2.nii').get_fdata()
# test_image_t2=scaler.fit_transform(test_image_t2.reshape(-1, test_image_t2.shape[-1])).reshape(test_image_t2.shape)

test_mask=nib.load('/data/UCSF-PDGM-0014_nifti/UCSF-PDGM-0014_tumor_segmentation.nii').get_fdata()
test_mask=test_mask.astype(np.uint8)

print(np.unique(test_mask))  #0, 1, 2, 4 (Need to reencode to 0, 1, 2, 3)
test_mask[test_mask==4] = 3  #Reassign mask values 4 to 3
print(np.unique(test_mask)) 


2563.422310464084
[0 1 2 4]
[0 1 2 3]


In [4]:
import os
import random

# Directory containing your data
data_directory = '/data'

# Exclude the specific path you want to skip
excluded_path = '/data/UCSF-PDGM-0541_nifti'
excluded_dirname = os.path.basename(excluded_path)
if excluded_path in data_directory:
    data_directory = data_directory.replace(excluded_path, '')

# List all files in the directory
all_files = os.listdir(data_directory)

# Exclude the specific path you want to skip
# excluded_path = '/data/UCSF-PDGM-0541_nifti'
# if excluded_path in all_files:
#     all_files.remove(excluded_path)

# Shuffle the list of files
random.shuffle(all_files)

# Define the number of files for training and validation
num_train_files = 400

# Split the shuffled list into training and validation sets
train_files = all_files[:num_train_files]
val_files = all_files[num_train_files:]

# Define the paths for training and validation datasets
TRAIN_DATASET_PATH = [os.path.join(data_directory, file) for file in train_files]
VAL_DATASET_PATH = [os.path.join(data_directory, file) for file in val_files]

# Print the first few file paths for verification
print("Training Dataset Paths:")
print(TRAIN_DATASET_PATH[:5])  # Print the first 5 paths
print("\nValidation Dataset Paths:")
print(VAL_DATASET_PATH[:5])    # Print the first 5 paths

Training Dataset Paths:
['/data/UCSF-PDGM-0416_nifti', '/data/UCSF-PDGM-0207_nifti', '/data/UCSF-PDGM-0285_nifti', '/data/UCSF-PDGM-0108_nifti', '/data/UCSF-PDGM-0409_FU001d_nifti']

Validation Dataset Paths:
['/data/UCSF-PDGM-0430_nifti', '/data/UCSF-PDGM-0190_nifti', '/data/UCSF-PDGM-0317_nifti', '/data/UCSF-PDGM-0027_nifti', '/data/UCSF-PDGM-0032_nifti']


In [5]:
train_t2_list = []
train_t1ce_list = []
train_flair_list = []
train_mask_list = []

# Iterate through all directories in TRAIN_DATASET_PATH
for directory in TRAIN_DATASET_PATH:
    # Extract the base directory name, removing '_nifti' if it is part of the directory name
    base_directory_name = os.path.basename(directory)
    
    # Assuming the ID is correctly represented in the directory name without '_nifti'
    ID = base_directory_name.replace('_nifti', '')

    # Construct file paths without '_nifti' in the filename part
    t2_path = glob.glob(os.path.join(directory, f'{ID}_T2.nii'))
    t1ce_path = glob.glob(os.path.join(directory, f'{ID}_T1c.nii'))
    flair_path = glob.glob(os.path.join(directory, f'{ID}_FLAIR.nii'))
    mask_path = glob.glob(os.path.join(directory, f'{ID}_tumor_segmentation.nii'))

    # Check if all modalities are present for the ID and add them at the same index
    if t2_path and t1ce_path and flair_path and mask_path:
        train_t2_list.append(t2_path[0])  
        train_t1ce_list.append(t1ce_path[0])
        train_flair_list.append(flair_path[0])
        train_mask_list.append(mask_path[0])

In [6]:
import os
import glob

# Initialize lists for different types of files
val_t2_list = []
val_t1ce_list = []
val_flair_list = []
val_mask_list = []

# Iterate through all directories in TRAIN_DATASET_PATH
for directory in VAL_DATASET_PATH:
    # Extract the base directory name, removing '_nifti' if it is part of the directory name
    base_directory_name = os.path.basename(directory)
    
    # Assuming the ID is correctly represented in the directory name without '_nifti'
    ID = base_directory_name.replace('_nifti', '')

    # Construct file paths without '_nifti' in the filename part
    t2_path = glob.glob(os.path.join(directory, f'{ID}_T2.nii'))
    t1ce_path = glob.glob(os.path.join(directory, f'{ID}_T1c.nii'))
    flair_path = glob.glob(os.path.join(directory, f'{ID}_FLAIR.nii'))
    mask_path = glob.glob(os.path.join(directory, f'{ID}_tumor_segmentation.nii'))

    # Check if all modalities are present for the ID and add them at the same index
    if t2_path and t1ce_path and flair_path and mask_path:
        val_t2_list.append(t2_path[0])  
        val_t1ce_list.append(t1ce_path[0])
        val_flair_list.append(flair_path[0])
        val_mask_list.append(mask_path[0])

In [7]:
index_to_check = 0  # Example index
if index_to_check < len(val_t2_list):
    print(f"Files at index {index_to_check}:")
    print(f"T2: {val_t2_list[index_to_check]}")
    print(f"T1ce: {val_t1ce_list[index_to_check]}")
    print(f"FLAIR: {val_flair_list[index_to_check]}")
    print(f"Mask: {val_mask_list[index_to_check]}")
else:
    print("Index is out of range.")

Files at index 0:
T2: /data/UCSF-PDGM-0430_nifti/UCSF-PDGM-0430_T2.nii
T1ce: /data/UCSF-PDGM-0430_nifti/UCSF-PDGM-0430_T1c.nii
FLAIR: /data/UCSF-PDGM-0430_nifti/UCSF-PDGM-0430_FLAIR.nii
Mask: /data/UCSF-PDGM-0430_nifti/UCSF-PDGM-0430_tumor_segmentation.nii


In [8]:
images_dir = '/exp1/experiment1Softmax/images'
masks_dir = '/exp1/experiment1Softmax/masks'

# Ensure the directories exist
os.makedirs(images_dir, exist_ok=True)
os.makedirs(masks_dir, exist_ok=True)

In [9]:
images_dir = '/exp1/experiment1Softmax/val/images'
masks_dir = '/exp1/experiment1Softmax/val/masks'

# Ensure the directories exist
os.makedirs(images_dir, exist_ok=True)
os.makedirs(masks_dir, exist_ok=True)

In [10]:
models = '/exp1/savedModels'
# masks_dir = '/exp1/new_model/experiment1Softmax/val/masks'

# Ensure the directories exist
os.makedirs(models, exist_ok=True)
# os.makedirs(masks_dir, exist_ok=True)

In [11]:
def load_img(img_dir, img_list):
    images = []
    for image_name in img_list:    
        if image_name.endswith('.npy'):
            image_path = os.path.join(img_dir, image_name)
            image = np.load(image_path)
            images.append(image)
    return np.array(images)

def imageLoader(img_dir, img_list, mask_dir, mask_list, batch_size):
    L = len(img_list)

    while True:  # Generator loop
        for batch_start in range(0, L, batch_size):
            limit = min(batch_start + batch_size, L)

            X = load_img(img_dir, img_list[batch_start:limit])
            Y = load_img(mask_dir, mask_list[batch_start:limit])

            yield (X, Y)  # Yielding a tuple with two elements

In [12]:
from matplotlib import pyplot as plt
import random

train_img_dir = "/exp1/experiment1Softmax/images/"
train_mask_dir = "/exp1/experiment1Softmax/masks/"
train_img_list = sorted(os.listdir(train_img_dir))
train_mask_list = sorted(os.listdir(train_mask_dir))


batch_size = 1

train_img_datagen = imageLoader(train_img_dir, train_img_list, train_mask_dir, train_mask_list, batch_size)